In [2]:
import Pkg
Pkg.add("JuMP")
import Pkg; Pkg.add("JSON")
using JSON
import Pkg
Pkg.add("HiGHS")

   Resolving package versions...


┌ Warning: The active manifest file at `C:\Data\2023-24\REOP\Kiro\Manifest.toml` has an old format that is being maintained.
│ To update to the new format, which is supported by Julia versions ≥ 1.6.2, run `import Pkg; Pkg.upgrade_manifest()` which will upgrade the format without re-resolving.
│ To then record the julia version re-resolve with `Pkg.resolve()` and if there are resolve conflicts consider `Pkg.update()`.
└ @ Pkg.Types C:\Users\caris\AppData\Local\Programs\Julia-1.9.4\share\julia\stdlib\v1.9\Pkg\src\manifest.jl:316


  No Changes to `C:\Data\2023-24\REOP\Kiro\Project.toml`
  No Changes to `C:\Data\2023-24\REOP\Kiro\Manifest.toml`


┌ Warning: The active manifest file is an older format with no julia version entry. Dependencies may have been resolved with a different julia version.
└ @ nothing C:\Data\2023-24\REOP\Kiro\Manifest.toml:0


   Resolving package versions...


  No Changes to `C:\Data\2023-24\REOP\Kiro\Project.toml`
  No Changes to `C:\Data\2023-24\REOP\Kiro\Manifest.toml`

   Resolving package versions...


  No Changes to `C:\Data\2023-24\REOP\Kiro\Project.toml`
  No Changes to `C:\Data\2023-24\REOP\Kiro\Manifest.toml`

In [3]:
# get functions

function get_parameters(name_json)
    general_parameters = name_json["general_parameters"]
    return general_parameters
end

function get_wind_turbines(name_json)
    get_wind_turbines = name_json["wind_turbines"]

    return get_wind_turbines
end

function get_wind_scenarios(name_json)
    get_wind_scenarios = name_json["wind_scenarios"]
    return get_wind_scenarios
end

function get_substation_locations(name_json)
    get_substation_locations = name_json["substation_locations"]
    
    return get_substation_locations
end

function get_substation_types(name_json)
    get_substation_types = name_json["substation_types"]

    return get_substation_types
end

function get_land_substation_cable_types(name_json)
    get_land_substation_cable_types = name_json["land_substation_cable_types"]

    return get_land_substation_cable_types
end

function get_substation_substation_cable_types(name_json)
    get_substation_substation_cable_types = name_json["substation_substation_cable_types"]

    return get_substation_substation_cable_types
end

get_substation_substation_cable_types (generic function with 1 method)

In [7]:
# data extraction

json_small = JSON.parsefile("KIRO-small.json")
file = json_small

P_max = get_parameters(file)["maximum_power"]
coords_mls = get_parameters(file)["main_land_station"]
C_ft = get_parameters(file)["fixed_cost_cable"]
C_lt = get_parameters(file)["variable_cost_cable"]
C_0 = get_parameters(file)["curtailing_cost"]
C_p = get_parameters(file)["curtailing_penalty"]
C_max = get_parameters(file)["maximum_curtailing"]
general_parameters = Dict("maximum_power" => P_max, "main_land_station" => coords_mls, "fixed_cost_cable" => C_ft, "variable_cost_cable" => C_lt, "curtailing_cost" => C_0, "curtailing_penalty" => C_p, "maximum_curtailing" => C_max)

wind_turbines = get_wind_turbines(file)
x_turbines = [turbine["x"] for turbine in wind_turbines]
y_turbines = [turbine["y"] for turbine in wind_turbines]
V_t = [x_turbines y_turbines]

wind_scenarios = get_wind_scenarios(file)

substation_locations = get_substation_locations(file)
x_substations = [substation["x"] for substation in substation_locations]
y_substations = [substation["y"] for substation in substation_locations]
V_s = [x_substations y_substations]

S = get_substation_types(file)

Q0 = get_land_substation_cable_types(file)

QS = get_substation_substation_cable_types(file)

print("done")

done

In [8]:
# Cost function in a linearized way



In [9]:
using HiGHS
using JuMP

# Create a JuMP model with HiGHS solver
model = Model(HiGHS.Optimizer)

# Define binary variables 
@variable(model, x[1:15, 1:20], binary=true)
@variable(model, y[1:113, 1:20], binary=true)
@variable(model, z[1:1125], binary=true)

# Set objective function 
@objective(model, Min, c(x,y,z))

# Constraint (1)
for v in 1:15
    @constraint(model, sum(x[v, s] for s in 1:20) <= 1)
end

# Constraint (2)
for v in 1:15
    S2 = sum(y[v, q] for q in 1:20)
    S1 = sum(x[v, s] for s in 1:20)
    @constraint(model, S1 == S2)
end

# Constraint (3)
for t in 1:75
    @constraint(model, sum(z[t+v] for v in 1:15) == 1)
end

# Constraint (4)
for v in 1:15
    S1 = sum(x[v, s] for s in 1:20)
    S2 = sum(sum(y[e, q] for q in 1:18) for e in 16:113)
    @constraint(model, S1 == S2)
end


# Solve the MILP problem
optimize!(model)


Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
120 rows, 2453 cols, 28785 nonzeros
0 rows, 2 cols, 0 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve: Optimal

Solving report
  Status            Optimal
  Primal bound      0
  Dual bound        0
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
                    0 (objective)
                    0 (bound viol.)
                    0 (int. viol.)
                    0 (row viol.)
  Timing            0.03 (total)
                    0.03 (presolve)
                    0.00 (postsolve)
  Nodes             0
  LP iterations     0 (total)
                    0 (strong br.)
                    0 (separation)
                    0 (heuristics)
